## --------------------------Testing Accuracy on Validation Dataset---------------------------- 

In [16]:
import nltk
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sb
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings("ignore")
import unidecode
#from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.stem import PorterStemmer
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import matplotlib.animation as animation
import operator
#import plotly.express as px
from collections import Counter
%matplotlib inline
import pandas_profiling

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [59]:
data=pd.read_csv("logistic_regression.csv")
#data=data.drop('tokenized',axis=1)
#data=data.drop('Unnamed: 0',axis=1)
data.head()

,Unnamed: 0,text,Tweet_tokenized,sentiment
0,0,last session day,"['last', 'session', 'day']",neutral
1,1,shanghai also realli excit precis skyscrap...,"['shanghai', 'also', 'realli', 'excit', 'preci...",neutral
2,2,recess hit veroniqu branquinho quit company ...,"['recess', 'hit', 'veroniqu', 'branquinho', 'q...",neutral
3,3,happi bday,"['happi', 'bday']",neutral
4,4,I like it,"['I', 'like', 'it']",neutral


In [60]:
data.shape

(3534, 4)

### Pre - Processing 

In [61]:
data.dropna
data.describe()

,Unnamed: 0
count,3534.000000
mean,1766.500000
std,1020.322253
min,0.000000
25%,883.250000
50%,1766.500000
75%,2649.750000
max,3533.000000


In [62]:
## Convert to lower case
data['text'] = data['text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
data['text'].head()

0                                     last session day
1    shanghai also realli excit precis skyscrap gal...
2    recess hit veroniqu branquinho quit company shame
3                                           happi bday
4                                            i like it
Name: text, dtype: object

In [63]:
## Stopwards Removal
data['text'] = data['text'].apply(lambda x : ' '.join([word for word in str(x).split() if not word in set(stopwords.words('english'))]))

In [64]:
## Lemmatization
lemmatizer = WordNetLemmatizer()
data['text'] = data['text'].apply(lambda x : ' '.join([lemmatizer.lemmatize(word) for word in str(x).split()]))

In [65]:
from textblob import TextBlob
data['text'][:5].apply(lambda x: str(TextBlob(x).correct()))

0                                     last session day
1    shanghai also really exit precise skyscrap gal...
2    recess hit veroniqu branquinho quit company shame
3                                            happy day
4                                                 like
Name: text, dtype: object

In [66]:
def clean_tweets(tweet):
    tweet = re.sub('http://[A-Za-z0-9./]+','',tweet) 
    # remove special characters 
    tweet = re.sub('[^ a-zA-Z0-9]', '', tweet)
    # remove Numbers
    tweet = re.sub('[0-9]', '', tweet)
    return tweet
data['text'] = data['text'].apply(clean_tweets)

In [9]:
def tokenize(text):
    return word_tokenize(text)
data['tokenized'] = data['text'].apply(tokenize)
data.head()

,Unnamed: 0,text,sentiment,tokenized
0,0,last session day httptwitpiccomezh,negative,"[last, session, day, httptwitpiccomezh]"
1,1,shanghai also realli excit precis skyscrap gal...,neutral,"[shanghai, also, realli, excit, precis, skyscr..."
2,2,recess hit veroniqu branquinho quit company shame,positive,"[recess, hit, veroniqu, branquinho, quit, comp..."
3,3,happi bday,neutral,"[happi, bday]"
4,4,httptwitpiccomwp like,positive,"[httptwitpiccomwp, like]"


### Feature Extraction using TF-IDF Vectorizer 

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer
# create the transform
cv = TfidfVectorizer(max_features=5000) 
x = cv.fit_transform(data['text'].tolist()).toarray()

In [68]:
data.sentiment[data.sentiment == 'positive'] = 1
data.sentiment[data.sentiment == 'neutral'] = 0
data.sentiment[data.sentiment == 'negative'] = 2
y=data['sentiment'].astype('float')
print(y)

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
3529    1.0
3530    0.0
3531    0.0
3532    0.0
3533    0.0
Name: sentiment, Length: 3534, dtype: float64


### Modelling 

In [69]:
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import expon,uniform,randint

#Sklearn imports
from sklearn import linear_model
from sklearn.model_selection import train_test_split,RandomizedSearchCV,cross_val_score,cross_val_predict,validation_curve
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [70]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0) #Basic train_test_split works here
y_test.shape

(884,)

In [48]:
from sklearn.svm import SVC
svc = SVC(kernel='linear')

In [49]:
svc.fit(x_train,y_train)

SVC(kernel='linear')

In [ ]:
y_pred=svc.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

### Random Forest Classifier 

In [74]:
from sklearn.ensemble import RandomForestClassifier
m = RandomForestClassifier(n_estimators=40, min_samples_leaf=3, n_jobs=-1)
m.fit(x_train, y_train)
predictions = m.predict(x_test)
print(classification_report(y_test, predictions))
print("Accuracy :",accuracy_score(y_test, predictions),'\n')

              precision    recall  f1-score   support

         0.0       0.84      0.99      0.91       710
         1.0       0.88      0.30      0.45        69
         2.0       0.72      0.17      0.28       105

    accuracy                           0.84       884
   macro avg       0.81      0.49      0.55       884
weighted avg       0.83      0.84      0.80       884

Accuracy : 0.8371040723981901 



### Logistic Regression 

In [72]:
# With single train test split
clf = LogisticRegression(solver='liblinear').fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8099547511312217
[[709   0   1]
 [ 66   1   2]
 [ 99   0   6]]
              precision    recall  f1-score   support

         0.0       0.81      1.00      0.90       710
         1.0       1.00      0.01      0.03        69
         2.0       0.67      0.06      0.11       105

    accuracy                           0.81       884
   macro avg       0.83      0.36      0.34       884
weighted avg       0.81      0.81      0.73       884



### Multinomial Naive Bayes 

In [73]:
## Multinomial NB with single train test split
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB().fit(x_train, y_train)

y_pred=nb_model.predict(x_test)

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8031674208144797
[[710   0   0]
 [ 69   0   0]
 [105   0   0]]
              precision    recall  f1-score   support

         0.0       0.80      1.00      0.89       710
         1.0       0.00      0.00      0.00        69
         2.0       0.00      0.00      0.00       105

    accuracy                           0.80       884
   macro avg       0.27      0.33      0.30       884
weighted avg       0.65      0.80      0.72       884

